# Food Rescuer

This code runs the Food Rescuer on Google Colab. The location of the project folder comes from my personal Google Drive, so to run the assistant on a different machine you'll need to specify the location of the food_rescuer folder


```
food_rescuer_path
recipes_path
```



In [ ]:
import sys
import os
from IPython.display import HTML, display, clear_output
from google.colab.output import eval_js
import IPython.display as ipd

# Set up system path
sys.path.append('/content/drive/MyDrive')  # Adjust this path as needed

# Find Food Rescuer project
food_rescuer_path = '/content/drive/MyDrive/food_rescuer'
if not os.path.exists(food_rescuer_path):
    print(f"Error: Project directory not found at {food_rescuer_path}")
    print("Please make sure the food_rescuer folder is in your Google Drive root.")
    raise FileNotFoundError(f"Directory not found: {food_rescuer_path}")
else:
    print(f"Found Food Rescuer project at {food_rescuer_path}")

# Import necessary modules
from food_rescuer.conversation.state_manager import ConversationManager
from food_rescuer.data.food_substitutions import SubstitutionKnowledgeBase
from food_rescuer.models.recipe_retrieval import RecipeRetriever
from food_rescuer.models.dl_intent_classifier import DeepLearningIntentClassifier

# Check if processed data exists
recipes_path = '/content/drive/MyDrive/food_rescuer/data/processed/processed_recipes.json'
if not os.path.exists(recipes_path):
    print(f"Warning: Processed recipes not found at {recipes_path}")
    print("Running data processing script...")

    # Run the data processing script
    %run "{food_rescuer_path}/data/process_data.py"
else:
    print(f"Found processed recipes at {recipes_path}")

# Initialize components
print("Initializing Food Rescuer assistant...")

# Load substitution knowledge base
print("Loading ingredient substitutions...")
substitution_kb = SubstitutionKnowledgeBase()

# Initialize recipe retriever
print("Loading recipe database...")
recipe_retriever = RecipeRetriever()

# Initialize deep learning intent classifier
print("Setting up deep learning intent classifier...")
intent_classifier = DeepLearningIntentClassifier()

# Create the conversation manager
conversation_manager = ConversationManager(
    substitution_kb=substitution_kb,
    recipe_retriever=recipe_retriever,
    intent_classifier=intent_classifier
)

print("Food Rescuer assistant initialized successfully!")

# Create a simple chat interface using HTML and JavaScript
js_code = """
function updateChat() {
  var chatbox = document.getElementById('chatbox');
  chatbox.scrollTop = chatbox.scrollHeight;
}

function setFocus() {
  document.getElementById('user_input').focus();
}

// Initialize
setTimeout(updateChat, 100);
setTimeout(setFocus, 200);
"""

# Clear the current output to have a clean slate for the UI
clear_output(wait=True)

# Store all messages in a list to re-render them
chat_messages = [
    {"role": "assistant", "content": "Hi there! What ingredients do you have today?"}
]

# Function to render the entire chat UI
def render_chat():
    # Display a welcome message
    welcome_html = """
    <div style="background-color: #f8f9fa; padding: 20px; border-radius: 10px; margin-bottom: 20px; text-align: center; color: #333; font-weight: normal;">
      <h1 style="color: #2c3e50; font-weight: bold;">🥕 FOOD RESCUER 🍳</h1>
      <h3 style="color: #2c3e50; font-weight: bold;">Cook better with what you have</h3>
      <p style="color: #333; font-weight: normal;">
        I'll help you find recipes using ingredients you already have,<br>
        and suggest substitutions when you're missing something.
      </p>
      <div style="text-align: left; max-width: 400px; margin: 0 auto; border-left: 3px solid #28a745; padding-left: 10px;">
        <strong style="color: #333;">Try saying:</strong>
        <ul style="color: #333; font-weight: normal;">
          <li>"I have eggs, flour, and milk"</li>
          <li>"What can I make with chicken and rice?"</li>
          <li>"I don't have butter, what can I use instead?"</li>
          <li>"Show me the recipe"</li>
          <li>"Next step"</li>
        </ul>
      </div>
    </div>
    """

    display(HTML(welcome_html))

    # Create chat messages HTML
    messages_html = ""
    for msg in chat_messages:
        if msg["role"] == "user":
            messages_html += f"""
            <div style="background-color: #d0e2f3; border-radius: 10px; padding: 10px; margin-bottom: 10px; text-align: right; border-right: 5px solid #4472c4; color: #333; font-weight: normal;">
              <strong style="color: #2c3e50;">You:</strong><br>{msg["content"]}
            </div>
            """
        else:
            messages_html += f"""
            <div style="background-color: #e2f0d9; border-radius: 10px; padding: 10px; margin-bottom: 10px; border-left: 5px solid #70ad47; color: #333; font-weight: normal;">
              <strong style="color: #2c3e50;">Food Rescuer:</strong><br>{msg["content"]}
            </div>
            """

    # Create and display the chat UI with fixed position for input
    chat_container = """
    <div style="max-width: 800px; margin: 0 auto; position: relative;">
      <div id="chatbox" style="height: 400px; border: 1px solid #ccc; padding: 10px; overflow-y: auto; background-color: #f9f9f9; border-radius: 5px; margin-bottom: 10px;">
        """ + messages_html + """
      </div>

      <div id="input_container" style="display: flex; position: sticky; bottom: 0; background-color: #f9f9f9; padding: 10px 0;">
        <input id="user_input" type="text" placeholder="What ingredients do you have?"
               style="flex-grow: 1; padding: 10px; border: 1px solid #666; border-radius: 5px; margin-right: 5px; color: #000;
                      background-color: #fff; font-weight: normal; font-size: 16px;">
        <button id="send_button"
                style="padding: 10px 15px; background-color: #4CAF50; color: white; border: none; border-radius: 5px; cursor: pointer; font-weight: bold;">
          Send
        </button>
      </div>
    </div>
    <script>
      // Focus on input field
      document.getElementById('user_input').focus();

      // Send message when Enter key is pressed
      document.getElementById('user_input').addEventListener('keypress', function(e) {
        if (e.key === 'Enter') {
          sendMessage();
        }
      });

      // Send message when button is clicked
      document.getElementById('send_button').addEventListener('click', sendMessage);

      function sendMessage() {
        var userInput = document.getElementById('user_input');
        var message = userInput.value.trim();

        if (message === '') return;  // Don't send empty messages

        // Clear input field
        userInput.value = '';

        // Return the message to Python for processing
        google.colab.kernel.invokeFunction('handleMessage', [message], {});
      }

      // Scroll to bottom of chat
      var chatbox = document.getElementById('chatbox');
      chatbox.scrollTop = chatbox.scrollHeight;
    </script>
    """

    display(HTML(chat_container))
    eval_js(js_code)

# Register a callback function for handling messages
from google.colab import output

# Function to handle user messages
def handle_message(message):
    # Add user message to chat history
    chat_messages.append({"role": "user", "content": message})

    # Get intent classification
    intent_data = intent_classifier.classify(message)
    print(f"[Debug] Intent: {intent_data['intent']} (confidence: {intent_data['confidence']:.4f})")

    # Process message through conversation manager
    response = conversation_manager.process(message)

    # Format the response by replacing newlines with <br>
    formatted_response = response.replace('\n', '<br>')

    # Add assistant message to chat history
    chat_messages.append({"role": "assistant", "content": formatted_response})

    # Re-render the entire chat
    clear_output(wait=True)
    render_chat()

# Register the function
output.register_callback('handleMessage', handle_message)

# Initial render of the chat interface
render_chat()

print("Chat interface is ready! Type your message in the input box.")